In [2]:
# Importing necessary libraries and modules for data analysis and visualization
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import sys
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm
import statsmodels.nonparametric.kernel_regression as loess
warnings.filterwarnings("ignore") # Disabling warnings to prevent cluttering the output
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
#Locate the folder
current_path = os.getcwd()
#import Data
data_all = pd.read_csv(current_path+ "\morg-2014-emp.csv")

In [7]:
#Reviewing the head of the Data
data_all.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.00,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent
2,6,75680310997590,January,AL,3936.9110,1090.00,60,41,2,NaN,...,2,6,"Native, Born In US",63,Motor vehicles and motor vehicle equipment man...,8140,"Private, For Profit",No,No,Employed-At Work
3,10,179140131100930,January,AL,3288.3640,769.23,40,40,1,NaN,...,2,4,"Native, Born In US",63,"**Publishing, except newspapers and software (...",8255,"Private, For Profit",Yes,NaN,Employed-At Work
4,11,179140131100930,January,AL,3422.8500,826.92,40,43,1,NaN,...,2,4,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",5940,"Private, For Profit",No,No,Employed-At Work


Choosing Biological scientist as the occupation of interest

In [8]:
#Setting the column called sample as 1 in cases where it is of the occupation of interest for us otherwise 0
data_all.loc[data_all["occ2012"] == 1610, "sample"] = 1
data_all.loc[data_all["sample"].isna(), "sample"] = 0

In [9]:
#seeing the description of our sample
data_all[data_all["sample"] == 1].describe()

,Unnamed: 0,hhid,weight,earnwke,uhours,grade92,race,ethnic,age,sex,marital,ownchild,chldpres,occ2012,sample
count,155.000000,1.550000e+02,155.000000,155.000000,155.000000,155.000000,155.000000,5.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.0,155.0
mean,160899.554839,4.446691e+14,1855.869986,1284.034645,41.651613,43.941935,1.341935,2.000000,42.993548,1.458065,2.851613,0.703226,1.703226,1610.0,1.0
std,94917.066970,3.179364e+14,1372.855396,654.399530,7.286880,1.275334,0.893255,1.224745,11.468476,0.499853,2.684234,1.039319,2.952380,0.0,0.0
min,416.000000,2.101078e+11,260.656100,150.000000,16.000000,39.000000,1.000000,1.000000,21.000000,1.000000,1.000000,0.000000,0.000000,1610.0,1.0
25%,79114.500000,1.371727e+14,496.157550,807.690000,40.000000,43.000000,1.000000,1.000000,34.500000,1.000000,1.000000,0.000000,0.000000,1610.0,1.0
50%,157370.000000,4.107100e+14,1478.810100,1195.650000,40.000000,44.000000,1.000000,2.000000,41.000000,1.000000,1.000000,0.000000,0.000000,1610.0,1.0
75%,246786.500000,7.390949e+14,3088.489100,1633.630000,40.000000,44.000000,1.000000,2.000000,53.000000,2.000000,7.000000,1.000000,3.000000,1610.0,1.0
max,316773.000000,9.820600e+14,5431.490700,2884.610000,80.000000,46.000000,4.000000,4.000000,64.000000,2.000000,7.000000,5.000000,14.000000,1610.0,1.0


Checking the number of observations with the occupation of interest and in comparison to the rest

In [10]:
#counting the number of 0 and 1 in column sample
data_all["sample"].value_counts()

sample
0.0    149161
1.0       155
Name: count, dtype: int64

Cleaning Data and working with on the missing values

In [12]:
#checking the columns with missing values
nan_columns = data_all.columns[data_all.isna().any()].tolist()
print(nan_columns)
# The number of missing observations under column ethnic
nan_count_ethnic = data_all["ethnic"].isna().sum()
print(nan_count_ethnic)
nan_count_unioncov = data_all["unioncov"].isna().sum()
print(nan_count_unioncov)

['ethnic', 'unioncov']
129245
17096


Given the number of missing values in relation to the total number of observations, we decided to drop these columns

In [13]:
#Dropping the columns with missing values
data_all = data_all.drop(columns=["ethnic","unioncov" ])

Focusing on the occupation of choice: Biological scientist

In [14]:
#Only keeping the observations with the occupation of our choice
data_all = data_all[data_all["sample"] == 1]

In [15]:
#dropping the column Sample as it does not have any function for us now
data_all = data_all.drop(columns=["sample"])

Here we check the type of values under each column. Object columns and categorical data need to be worked on. 

In [16]:
#checking the types of the values under all columns
print(data_all.dtypes)

Unnamed: 0      int64
hhid            int64
intmonth       object
stfips         object
weight        float64
earnwke       float64
uhours          int64
grade92         int64
race            int64
age             int64
sex             int64
marital         int64
ownchild        int64
chldpres        int64
prcitshp       object
state          object
ind02          object
occ2012         int64
class          object
unionmme       object
lfsr94         object
dtype: object


In [17]:
#checking all possible values of column intmonth
unique_values = data_all["intmonth"].unique()
print(unique_values)

['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']


In [18]:
#producing dummy variables for months  in columns intmonth, and and US states in stfips, and other variables such as industry, calss, union membership and coverage, employment status and State
data_all_adjusted = pd.get_dummies(data_all, columns=["intmonth", "stfips", "state", "ind02", "class","unionmme", "lfsr94"], drop_first=True)


In [19]:
#checking all possible values of column prcitshp
unique_values = data_all["prcitshp"].unique()
print(unique_values)

['Foreign Born, Not a US Citizen' 'Native, Born In US'
 'Foreign Born, US Cit By Naturalization'
 'Native, Born Abroad Of US Parent(s)'
 'Native, Born in PR or US Outlying Area']


For this variable, only the issue of US citizenship is of interest for us

In [20]:
#we make a dummy based on citizenship
data_all_adjusted["native_dummy"] = data_all_adjusted["prcitshp"].isin([
    "Native, Born In US",
    "Native, Born Abroad Of US Parent(s)",
    "Native, Born in PR or US Outlying Area"
]).astype(int)
data_all_adjusted = data_all_adjusted.drop(columns=["prcitshp"])

In [21]:
#we want to transform the column race because in the current way it is not meaningful
data_all_adjusted = pd.get_dummies(data_all_adjusted, columns=["race"], drop_first=True)

In [22]:
print(data_all_adjusted.columns.tolist())

['Unnamed: 0', 'hhid', 'weight', 'earnwke', 'uhours', 'grade92', 'age', 'sex', 'marital', 'ownchild', 'chldpres', 'occ2012', 'intmonth_August', 'intmonth_December', 'intmonth_February', 'intmonth_January', 'intmonth_July', 'intmonth_June', 'intmonth_March', 'intmonth_May', 'intmonth_November', 'intmonth_October', 'intmonth_September', 'stfips_AR', 'stfips_AZ', 'stfips_CA', 'stfips_CO', 'stfips_CT', 'stfips_DC', 'stfips_DE', 'stfips_FL', 'stfips_GA', 'stfips_HI', 'stfips_IA', 'stfips_ID', 'stfips_IL', 'stfips_IN', 'stfips_KS', 'stfips_MA', 'stfips_MD', 'stfips_ME', 'stfips_MN', 'stfips_MO', 'stfips_MS', 'stfips_MT', 'stfips_NC', 'stfips_ND', 'stfips_NE', 'stfips_NH', 'stfips_NJ', 'stfips_NM', 'stfips_NV', 'stfips_NY', 'stfips_OH', 'stfips_OR', 'stfips_PA', 'stfips_RI', 'stfips_SD', 'stfips_TX', 'stfips_UT', 'stfips_VA', 'stfips_VT', 'stfips_WA', 'stfips_WY', 'state_2', 'state_3', 'state_4', 'state_5', 'state_6', 'state_7', 'state_8', 'state_9', 'state_1', 'state_2', 'state_3', 'state_4'

In [23]:
#calculate earnings per hour
data_all_adjusted["w"] = data_all_adjusted["earnwke"] / data_all_adjusted["uhours"]

Based on the informed decisions about the context, some transformations of the variables are also included

In [24]:
#Transformations and possible interaction terms
data_all_adjusted["age_sq"] = data_all_adjusted["age"] ** 2
data_all_adjusted["ln_age"] = np.log(data_all_adjusted["age"])
data_all_adjusted["ln_uhours"] = np.log(data_all_adjusted["uhours"])
data_all_adjusted["uhours_sq"] = data_all_adjusted["uhours"] ** 2
data_all_adjusted["ln_weight"] = np.log(data_all_adjusted["weight"])
data_all_adjusted["weight_sq"] = data_all_adjusted["weight"] ** 2
data_all_adjusted["age_weight_interaction"] = data_all_adjusted["age"] * data_all_adjusted["weight"]
data_all_adjusted["age_sex_interaction"] = data_all_adjusted["age"] * data_all_adjusted["sex"]

'c:\\Users\\behna\\Desktop\\project\\Data-Analysis-3'